In [1]:
import numpy as np
import pandas as pd
import random
import math,time,sys
from matplotlib import pyplot
from datetime import datetime
#==================================================================
def sigmoid1(gamma):     #convert to probability
	if gamma < 0:
		return 1 - 1/(1 + math.exp(gamma))
	else:
		return 1/(1 + math.exp(-gamma))

def sigmoid1i(gamma):     #convert to probability
	gamma = -gamma
	if gamma < 0:
		return 1 - 1/(1 + math.exp(gamma))
	else:
		return 1/(1 + math.exp(-gamma))

def sigmoid2(gamma):
	gamma /= 2
	if gamma < 0:
		return 1 - 1/(1 + math.exp(gamma))
	else:
		return 1/(1 + math.exp(-gamma))

def sigmoid3(gamma):
	gamma /= 3
	if gamma < 0:
		return 1 - 1/(1 + math.exp(gamma))
	else:
		return 1/(1 + math.exp(-gamma))

def sigmoid4(gamma):
	gamma *= 2
	if gamma < 0:
		return 1 - 1/(1 + math.exp(gamma))
	else:
		return 1/(1 + math.exp(-gamma))


def Vfunction1(gamma):
	return abs(np.tanh(gamma))

def Vfunction2(gamma):
	val = (math.pi)**(0.5)
	val /= 2
	val *= gamma
	val = math.erf(val)
	return abs(val)

def Vfunction3(gamma):
	val = 1 + gamma*gamma
	val = math.sqrt(val)
	val = gamma/val
	return abs(val)

def Vfunction4(gamma):
	val=(math.pi/2)*gamma
	val=np.arctan(val)
	val=(2/math.pi)*val
	return abs(val)

def initialize(popSize,dim):
	population=np.zeros((popSize,dim))
	minn = 1
	maxx = math.floor(0.8*dim)
	if maxx<minn:
		minn = maxx

	for i in range(popSize):
		random.seed(i**3 + 10 + time.time() ) 
		no = random.randint(minn,maxx)
		if no == 0:
			no = 1
		random.seed(time.time()+ 100)
		pos = random.sample(range(0,dim-1),no)
		for j in pos:
			population[i][j]=1

		# print(population[i])  
	return population

def fitness(solution, trainX, testX, trainy, testy):
	cols=np.flatnonzero(solution)
	val=1
	if np.shape(cols)[0]==0:
		return val	
	clf=RandomForestClassifier(n_estimators=10)
	train_data=trainX[:,cols]
	test_data=testX[:,cols]
	clf.fit(train_data,trainy)
	val=1-clf.score(test_data,testy)

	#in case of multi objective  []
	set_cnt=sum(solution)
	set_cnt=set_cnt/np.shape(solution)[0]
	val=omega*val+(1-omega)*set_cnt
	return val

def allfit(population, trainX, testX, trainy, testy):
	x=np.shape(population)[0]
	acc=np.zeros(x)
	for i in range(x):
		acc[i]=fitness(population[i],trainX,testX,trainy,testy)     
		#print(acc[i])
	return acc

def toBinary(solution,dimension,trainX,testX,trainy,testy):
	# print("continuous",solution)

	Xnew = np.zeros(np.shape(solution))
	for i in range(dimension):
		temp = sigmoid1(solution[i])

		random.seed(time.time()+i)
		if temp > 0.5: # sfunction
			Xnew[i] = 1
		else:
			Xnew[i] = 0
		# if temp > 0.5: # vfunction
		# 	Xnew[i] = 1 - solution[i]
		# else:
		# 	Xnew[i] = solution[i]

	return Xnew


#============================================================================
def socialmimic(dataset,popSize,maxIter):

	#--------------------------------------------------------------------
	df=pd.read_csv(dataset)
	(a,b)=np.shape(df)
	#print(a,b)
	data = df.values[:,0:b-1]
	label = df.values[:,b-1]
	dimension = np.shape(data)[1] #solution dimension
	#---------------------------------------------------------------------

	cross = 5
	test_size = (1/cross)
	trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size)


	clf=RandomForestClassifier(n_estimators=10)
	clf.fit(trainX,trainy)
	val=clf.score(testX,testy)
	whole_accuracy = val
	#print("Total Acc: ",val)

	x_axis = []
	y_axis = []
	population = initialize(popSize,dimension)
	GBESTSOL = np.zeros(np.shape(population[0]))
	GBESTFIT = 1000

	start_time = datetime.now()

	for currIter in range(1,maxIter):
		newpop = np.zeros((popSize,dimension))
		fitList = allfit(population,trainX,testX,trainy,testy)
		y_axis.append(min(fitList))
		x_axis.append(currIter)
		bestInx = np.argmin(fitList)
		fitBest = min(fitList)
		Xbest = population[bestInx].copy()

		if fitBest<GBESTFIT: 
			GBESTFIT = fitBest
			GBESTSOL = Xbest.copy()
			#print("",GBESTSOL.sum())

		for i in range(popSize):
			currFit = fitList[i]
			# print(currFit)
			difference = ( currFit - fitBest ) / currFit
			if difference == 0:
				random.seed(time.time())
				difference = random.uniform(0,1)
			newpop[i] = np.add(population[i],np.multiply(difference,population[i]))
			newpop[i] = toBinary(population[i],dimension,trainX,testX,trainy,testy)

		population = newpop.copy()
	# pyplot.plot(x_axis,y_axis)
	# pyplot.show()

	#test accuracy
	cols = np.flatnonzero(GBESTSOL)
	val = 1
	if np.shape(cols)[0]==0:
		return GBESTSOL
	clf = RandomForestClassifier(n_estimators=10)
	train_data = trainX[:,cols]
	test_data = testX[:,cols]
	clf.fit(train_data,trainy)
	val = clf.score(test_data,testy)
	return GBESTSOL,val


#========================================================================================================
datasetList = [""]

for dataset in datasetList:
	accuList = []
	featList = []
	for count in range(*):
		if (dataset == "*") and count>800:
			break
		#print(count)
		answer,testAcc = socialmimic("D:/OrtakCalismalar/Mesut/COVID/dataset/stacking/mobnet/bes_k_fold/"+dataset+".csv",maxIter)
		print(*)
		accuList.append(testAcc)
		print(featList.append(answer.sum()))
	inx = np.argmax(accuList)
	best_accuracy = accuList[inx]
	best_features = featList[inx]
	print(dataset,"best:",accuList[inx],featList[inx])

	with open("D:/OrtakCalismalar/Mesut/COVID/dataset/stacking/mobnet/bes_k_fold/result_FinalData.csv","a") as f:
		print(dataset,"%.2f" % (100*best_accuracy),best_no_features,file=f)

484.0
271.0
243.0
158.0
113.0
189.0
167.0
207.0
18.0
217.0
129.0
90.0
156.0
114.0
142.0
212.0
85.0
87.0
187.0
58.0
71.0
209.0
38.0
56.0
317.0
100.0
75.0
275.0
284.0
389.0
90.0
156.0
83.0
103.0
164.0
153.0
85.0
381.0
80.0
197.0
221.0
167.0
334.0
92.0
329.0
112.0
340.0
215.0
80.0
321.0
86.0
133.0
677.0
116.0
132.0
47.0
131.0
96.0
241.0
399.0
364.0
107.0
143.0
411.0
218.0
38.0
132.0
207.0
200.0
218.0
356.0
167.0
60.0
96.0
158.0
110.0
337.0
152.0
168.0
79.0
255.0
85.0
167.0
69.0
161.0
146.0
444.0
101.0
73.0
194.0
271.0
215.0
173.0
96.0
194.0
338.0
116.0
95.0
166.0
398.0
66.0
145.0
73.0
301.0
56.0
245.0
300.0
234.0
51.0
227.0
179.0
174.0
80.0
370.0
217.0
209.0
77.0
435.0
125.0
128.0
336.0
181.0
205.0
47.0
67.0
225.0
132.0
121.0
219.0
66.0
148.0
126.0
227.0
36.0
178.0
122.0
195.0
145.0
422.0
90.0
28.0
174.0
122.0
265.0
266.0
157.0
135.0
195.0
263.0
188.0
84.0
85.0
98.0
220.0
159.0
72.0
292.0
343.0
232.0
85.0
289.0
115.0
459.0
187.0
35.0
310.0
143.0
101.0
309.0
237.0
58.0
279.0
24.0
223.0
113

IndexError: list index out of range